# 🚀 Mapperatorinator LoRA Training

Notebook này được thiết kế để bạn có thể training model Mapperatorinator bằng phương pháp LoRA trên Google Colab GPU.

### 📝 Quy trình thực hiện:
1. **Chuẩn bị dữ liệu**: Đảm bảo bạn đã upload file `quality_dataset.zip` lên Google Drive.
2. **Cấu hình Runtime**: Chọn **Runtime -> Change runtime type -> GPU -> T4 GPU**.
3. **Chạy từng cell**: Nhấn nút Play (▶️) ở cạnh trái mỗi ô code.

---

## Bước 1: Kết nối Google Drive và chuẩn bị môi trường

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Clone repository
!git clone https://github.com/OliBomby/Mapperatorinator.git
%cd Mapperatorinator

# Cài đặt các thư viện cần thiết
!pip install -r requirements.txt
!pip install peft accelerate

## Bước 2: Giải nén Dataset từ Google Drive
*Lưu ý: Bạn hãy kiểm tra xem file `.zip` của bạn nằm ở đâu trên Drive và sửa đường dẫn nếu cần.*

In [ ]:
!mkdir -p datasets

# Đường dẫn file zip trên Drive (Mặc định là thư mục gốc Drive)
PATH_TO_ZIP = '/content/drive/MyDrive/quality_dataset.zip' 

import os
if os.path.exists(PATH_TO_ZIP):
    !unzip {PATH_TO_ZIP} -d datasets/
    print("✅ Đã giải nén dataset thành công!")
else:
    print("❌ Không tìm thấy file zip tại: ", PATH_TO_ZIP)
    print("Hãy đảm bảo bạn đã upload file quality_dataset.zip lên Google Drive.")

## Bước 3: Bắt đầu Training LoRA

In [ ]:
# Chạy lệnh training với các thông số tối ưu cho GPU T4 (16GB VRAM)
!python osuT5/train.py -cn lora \
    pretrained_path="TheBlueSkyARL/Mapperatorinator-v30" \
    data.train_dataset_path="datasets/quality_dataset" \
    data.test_dataset_path="datasets/quality_dataset" \
    data.train_dataset_end=150 \
    data.test_dataset_start=150 \
    data.test_dataset_end=166 \
    optim.batch_size=8 \
    optim.grad_acc=4 \
    optim.total_steps=2000 \
    precision="fp16" \
    compile=False

## ✅ Hoàn tất!
Sau khi chạy xong, các file trọng số LoRA của bạn sẽ nằm trong thư mục `outputs/`. 
Bạn có thể nén thư mục này lại và tải về máy tính để sử dụng.